# FARLAB - Robotable Streets Project 
Developer: @mattwfranchi

Project Members: Matt Franchi, Maria-Teresa Parreira, Frank Bu, Wendy Ju 

As robots deployments become more common, they will become yet another dancer in the sidewalk ballet. Within urban mapping, transit mobility and walkability scores have emerged as a way to measure the quality of a city's infrastructure for a specific medium of traffic. However, there is no such metric for robots! Here, we aim to envision what a 'robotability' score might look like, and how it might be used to inform urban planning and policy. 

We utilize the following data features in computing a *robotability score*: 
- Sidewalk width 
- Sidewalk quality proxied by 311 complaints 
- Pedestrian density, computed via aggregated dashcam data
- Sidewalk material (concrete, asphalt, cobblestone, etc.) 
- Connectivity: cellular coverage, WiFi availbility, IoT network coverage, and GPS coverage 
- Elevation change from beginning to end of road segment 
- Solar radiation levels, for potential solar charging and for potential overheating. 
- Proximity to hypothetical charging stations 
- Grating on sidewalk (ie in NYC, the subway grates) that might be problematic for robots to navigate 
- Snow buildup 
- Local attitudes towards robots 
- Average illegal parking levels, ie cars parked on sidewalks 
- Shade / shadows 
- Overhead covering (scaffolding, awnings, etc., in the case of non-waterproof bots)
- Zoning. My hypothesis: robots are more acceptable in commercial-zoned areas, and less acceptable in majority-residential zoned areas. 

### Other Things to Lock In (4/25/24): 
- Study period. Some of this data (311 complaints, pedestrian densities, etc., should be constrained within a time range. **Limitation: we don't have new, free dashcam data presently**)



In [39]:
# class RobotabilityGraph that inherits from Graph class 
import os
import sys 
sys.path.append("/share/ju/urban-fingerprinting")

import osmnx as ox 
import geopandas as gpd 
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
# enable latex plotting 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from glob import glob 
from tqdm import tqdm 

from shapely import wkt, LineString 

import rasterio
from rasterio.enums import Resampling
from rasterio.plot import show 


from src.utils.logger import setup_logger 

logger = setup_logger('robotability-score')
logger.setLevel("INFO")
logger.info("Modules initialized.")

WGS='EPSG:4326'
PROJ='EPSG:2263'

REGEN_SEGMENTIZATION=True
REGEN_TOPOLOGY=True

GEN_INSPECTION_PLOTS=True
INSPECTION_PLOTS="figures/inspection_plots"

os.makedirs(INSPECTION_PLOTS, exist_ok=True)


2024-08-21 22:02:43 - robotability-score - INFO - Modules initialized.


## Loading and Preprocessing Data Features 

### Neighborhood Tabulation Areas (NYC)

In [40]:
# Load the Neighborhood Tabulation Areas (NTAs) dataset 
ntas_nyc = pd.read_csv("data/ntas_nyc.csv")
ntas_nyc = gpd.GeoDataFrame(ntas_nyc, geometry=wkt.loads(ntas_nyc['the_geom']), crs=WGS).to_crs(PROJ)

# Remove redundant columns 
TO_DROP = ['BoroCode','CountyFIPS','NTA2020','NTAAbbrev','CDTA2020','CDTAName']
ntas_nyc = ntas_nyc.drop(columns=TO_DROP)

logger.success("NTAs loaded.")

2024-08-21 22:02:43 - robotability-score - SUCCESS - NTAs loaded.


### Census Blocks (NYC)

In [41]:
cbs_nyc = gpd.read_file("data/nycb2020_24c/nycb2020.shp")

TO_DROP = ['BoroCode', 'CT2020', 'BCTCB2020']

cbs_nyc = cbs_nyc.drop(columns=TO_DROP)
cbs_nyc = cbs_nyc.to_crs(PROJ)

logger.success("Census Blocks loaded.")

2024-08-21 22:02:49 - robotability-score - SUCCESS - Census Blocks loaded.


### Community Districts (NYC)

In [42]:
cds_nyc = gpd.read_file("data/community_districts_nyc/geo_export_d18ab225-a18b-4d26-8ff1-78a889568090.shp").to_crs(PROJ)
cds_nyc['boro_cd'] = cds_nyc['boro_cd'].astype(int)

### Sidewalk Basemap (NYC)

In [43]:
# Load the NYC sidewalk basemap 
sidewalk_nyc = pd.read_csv("data/sidewalks_nyc.csv")
sidewalk_nyc = gpd.GeoDataFrame(sidewalk_nyc, geometry=wkt.loads(sidewalk_nyc['the_geom']), crs=WGS).to_crs(PROJ)

In [44]:
# Take out features we don't need, and add a width column 
TO_DROP = ['SUB_CODE', 'FEAT_CODE', 'STATUS', 'the_geom']
sidewalk_nyc = sidewalk_nyc.drop(columns=TO_DROP)
sidewalk_nyc['SHAPE_Width'] = sidewalk_nyc['SHAPE_Area'] / sidewalk_nyc['SHAPE_Leng']

# Simplify 
sidewalk_nyc['geometry'] = sidewalk_nyc['geometry'].simplify(10)

# write to disk 
if REGEN_SEGMENTIZATION:
    # segmentize 
    segmentized = sidewalk_nyc.segmentize(50).extract_unique_points().explode(index_parts=True)

    segmentized = gpd.GeoDataFrame(segmentized).reset_index() 

    segmentized = segmentized.merge(sidewalk_nyc,left_on='level_0',right_index=True).drop(columns=['level_0','level_1','geometry'])
    segmentized['geometry'] = segmentized.iloc[:,0]
    segmentized.drop(segmentized.columns[0],axis=1, inplace=True)
    segmentized = gpd.GeoDataFrame(segmentized, crs=PROJ)

    # random id for each point in 'point_index' column 
    segmentized['point_index'] = segmentized.index

    segmentized.to_csv("data/sidewalks_nyc_segmentized.csv")
    logger.success("Segmentized sidewalk basemap written to disk.")

else: 
    segmentized = pd.read_csv("data/sidewalks_nyc_segmentized.csv")
    segmentized = gpd.GeoDataFrame(segmentized, geometry=wkt.loads(segmentized['geometry']), crs=PROJ)
    logger.info("Segmentized sidewalk basemap loaded.")


sidewalk_nyc = segmentized

logger.success("NYC sidewalk basemap loaded.")
logger.info(f"Distribution of sidewalk widths [ft]: \n{sidewalk_nyc['SHAPE_Width'].describe()}")

2024-08-21 22:04:04 - robotability-score - SUCCESS - Segmentized sidewalk basemap written to disk.
2024-08-21 22:04:04 - robotability-score - SUCCESS - NYC sidewalk basemap loaded.
2024-08-21 22:04:04 - robotability-score - INFO - Distribution of sidewalk widths [ft]: 
count    2.551220e+06
mean     5.373536e+00
std      1.480770e+00
min      2.710948e-01
25%      4.458703e+00
50%      5.149090e+00
75%      6.177234e+00
max      4.021491e+01
Name: SHAPE_Width, dtype: float64


In [45]:
sidewalk_nyc_COPY = sidewalk_nyc.copy()

In [46]:
if GEN_INSPECTION_PLOTS:
    fig, ax = plt.subplots(1,1,figsize=(10,10))

    ntas_nyc.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.5)
    sidewalk_nyc.sample(frac=0.1).plot(ax=ax, color='black', alpha=0.5, markersize=0.5)

    plt.title("NYC Sidewalk Basemap")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"{INSPECTION_PLOTS}/nyc_sidewalk_basemap.png")
    plt.close()

###  Topology 

In [47]:
TOPOGRAPHY_NYC = "data/1ft_dem_nyc/DEM_LiDAR_1ft_2010_Improved_NYC_int.tif"

downsample_factor = 10

# Open the raster
with rasterio.open(TOPOGRAPHY_NYC) as src:
    # Calculate new transform and dimensions
    new_transform = src.transform * src.transform.scale(
        downsample_factor,
        downsample_factor
    )
    new_width = src.width // downsample_factor
    new_height = src.height // downsample_factor
    
    # Resample the raster
    topology = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.bilinear
    )

    # Create a new rasterio-like object with updated metadata
    new_meta = src.meta.copy()
    new_meta.update({
        "driver": "GTiff",
        "height": new_height,
        "width": new_width,
        "transform": new_transform
    })

    # Write the new raster to disk
    if REGEN_TOPOLOGY: 
        with rasterio.open("data/1ft_dem_nyc/downsampled_topography.tif", "w", **new_meta) as dst:
            dst.write(topology)



In [48]:
# plot topology 
if GEN_INSPECTION_PLOTS:
    fig, ax = plt.subplots(figsize=(10, 10))
    fig.suptitle(r"\bf Elevation in New York City", fontsize=20)

    # elevation is a 2D array, so we can plot it directly
    show(topology, ax=ax, cmap='terrain', transform=new_transform, )
    #nyc_ct.plot(ax=ax, facecolor='white', edgecolor='black', linewidth=0.5, alpha=0.25)

    ax.set_axis_off()
    plt.savefig(f"{INSPECTION_PLOTS}/topology_nyc.png", dpi=300)
    plt.close()

In [49]:
import rasterio
from itertools import islice
from tqdm import tqdm

def chunked_iterable(iterable, size):
    """Yield successive n-sized chunks from the iterable."""
    it = iter(iterable)
    while chunk := list(islice(it, size)):
        yield chunk

# Get elevation at proj_x, proj_y
sampled_elevations = []
chunk_size = 1000

with rasterio.open("data/1ft_dem_nyc/downsampled_topography.tif") as src:
    # Generate iterable of (x, y) tuples from proj_x and proj_y columns
    xy = zip(sidewalk_nyc.geometry.x, sidewalk_nyc.geometry.y)
    total_points = len(sidewalk_nyc)

    # Process in chunks with progress bar
    for chunk in tqdm(chunked_iterable(xy, chunk_size), total=(total_points // chunk_size) + 1, desc="Processing Elevations"):
        chunk_elevations = rasterio.sample.sample_gen(src, chunk)
        sampled_elevations.extend(list(chunk_elevations))

# Convert sampled elevations to a list and add to the DataFrame
sidewalk_nyc['ft_above_sea'] = list(map(lambda x: x[0], sampled_elevations))
print(sidewalk_nyc.info())

Processing Elevations: 100%|██████████| 2552/2552 [01:04<00:00, 39.39it/s]


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2551220 entries, 0 to 2551219
Data columns (total 7 columns):
 #   Column        Dtype   
---  ------        -----   
 0   SOURCE_ID     int64   
 1   SHAPE_Leng    float64 
 2   SHAPE_Area    float64 
 3   SHAPE_Width   float64 
 4   geometry      geometry
 5   point_index   int64   
 6   ft_above_sea  uint16  
dtypes: float64(3), geometry(1), int64(2), uint16(1)
memory usage: 121.7 MB
None


In [50]:
sidewalk_nyc['ft_above_sea'].describe() 

count    2.551220e+06
mean     5.448646e+01
std      4.528154e+01
min      0.000000e+00
25%      2.100000e+01
50%      4.200000e+01
75%      7.400000e+01
max      3.920000e+02
Name: ft_above_sea, dtype: float64

### Satellite Availability 

In [51]:
gso_satellite = pd.read_csv("data/bdc_36_GSOSatellite_fixed_broadband_D23_06aug2024.csv", engine='pyarrow')

gso_satellite['block_geoid'] = gso_satellite['block_geoid'].astype(str)

In [52]:
# merge gso_satellite with cbs_nyc 
gso_satellite = cbs_nyc.merge(gso_satellite, right_on='block_geoid', left_on='GEOID', how='left')
gso_satellite = gpd.GeoDataFrame(gso_satellite, crs=PROJ)
logger.info(f"GSOSatellite data merged with Census Blocks. \n{gso_satellite.info()}")

2024-08-21 22:05:55 - robotability-score - INFO - GSOSatellite data merged with Census Blocks. 
None


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2470523 entries, 0 to 2470522
Data columns (total 18 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   CB2020                         object  
 1   BoroName                       object  
 2   GEOID                          object  
 3   Shape_Leng                     float64 
 4   Shape_Area                     float64 
 5   geometry                       geometry
 6   frn                            float64 
 7   provider_id                    float64 
 8   brand_name                     object  
 9   location_id                    float64 
 10  technology                     float64 
 11  max_advertised_download_speed  float64 
 12  max_advertised_upload_speed    float64 
 13  low_latency                    float64 
 14  business_residential_code      object  
 15  state_usps                     object  
 16  block_geoid                    object  
 17  h3_res8_id         

In [53]:
ngso_satellite = pd.read_csv("data/bdc_36_NGSOSatellite_fixed_broadband_D23_06aug2024.csv", engine='pyarrow')
ngso_satellite['block_geoid'] = ngso_satellite['block_geoid'].astype(str)

# merge ngso_satellite with cbs_nyc
ngso_satellite = cbs_nyc.merge(ngso_satellite, right_on='block_geoid', left_on='GEOID', how='left')
ngso_satellite = gpd.GeoDataFrame(ngso_satellite, crs=PROJ)
logger.info(f"NSGOSatellite data merged with Census Blocks. \n{ngso_satellite.info()}")

2024-08-21 22:05:59 - robotability-score - INFO - NSGOSatellite data merged with Census Blocks. 
None


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 826917 entries, 0 to 826916
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   CB2020                         826917 non-null  object  
 1   BoroName                       826917 non-null  object  
 2   GEOID                          826917 non-null  object  
 3   Shape_Leng                     826917 non-null  float64 
 4   Shape_Area                     826917 non-null  float64 
 5   geometry                       826917 non-null  geometry
 6   frn                            821993 non-null  float64 
 7   provider_id                    821993 non-null  float64 
 8   brand_name                     821993 non-null  object  
 9   location_id                    821993 non-null  float64 
 10  technology                     821993 non-null  float64 
 11  max_advertised_download_speed  821993 non-null  float64 
 12  max_adve

### Surveillance Cameras 

In [54]:
surveillance_cameras = pd.read_csv("data/surveillance_cameras/data/cameras_2015-2021.csv")
surveillance_cameras = surveillance_cameras[surveillance_cameras['city'] == 'New York']
surveillance_cameras = surveillance_cameras[surveillance_cameras['camera_count'] > 0]
surveillance_cameras 

/tmp/ipykernel_2411866/4147868696.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  surveillance_cameras = pd.read_csv("data/surveillance_cameras/data/cameras_2015-2021.csv")


,panoid,heading,lat,lon,city,year,month,distance,camera_count,zone_type
4152,qbafD92lLAk_SRn2GErWkg,84,40.549514,-74.151841,New York,2018,7,4.905160,1,commercial
4298,YaZUN1_WQa2mt6ZHU8f0IA,145,40.550640,-74.197978,New York,2018,8,2.886381,1,residential
5835,2Rh9tnnR1KrW1Xjk20zRlg,215,40.562025,-74.101158,New York,2018,6,13.419243,1,residential
6481,0jNVCy-Er4yEhGr5oBwNkw,31,40.568562,-74.106132,New York,2017,7,4.582663,1,residential
7335,rt7iOq7usJr3YM3huiCY1g,212,40.575047,-74.165660,New York,2015,9,7.510337,1,commercial
...,...,...,...,...,...,...,...,...,...,...
98816,dspn02Xm1hsWuvUNi_JDoA,115,40.890967,-73.828891,New York,2018,8,3.015696,1,residential
99023,KDtTu6amyjs-VaMRwBnoDg,202,40.893688,-73.860541,New York,2016,7,4.943950,1,residential
99028,YUU9BMH1B_cinU2UWSlWXQ,203,40.893713,-73.858487,New York,2016,7,5.639436,1,residential
99364,ejcY-mgRFhaooy5h_4rzRw,352,40.898056,-73.862961,New York,2016,7,11.193857,1,residential


### Street Furniture 

In [55]:
street_furniture_nyc = pd.read_csv("data/processed/street_furniture_density.csv", engine='pyarrow')

logger.success("Street furniture density data loaded.")

2024-08-21 22:06:12 - robotability-score - SUCCESS - Street furniture density data loaded.


In [56]:
sidewalk_nyc 

,SOURCE_ID,SHAPE_Leng,SHAPE_Area,SHAPE_Width,geometry,point_index,ft_above_sea
0,2380000336,3504.153579,19782.066985,5.645320,POINT (1000940.908 165223.488),0,19
1,2380000336,3504.153579,19782.066985,5.645320,POINT (1000948.337 165175.884),1,20
2,2380000336,3504.153579,19782.066985,5.645320,POINT (1000955.765 165128.280),2,20
3,2380000336,3504.153579,19782.066985,5.645320,POINT (1000963.194 165080.677),3,19
4,2380000336,3504.153579,19782.066985,5.645320,POINT (1000970.623 165033.073),4,20
...,...,...,...,...,...,...,...
2551215,9380001047,1301.694419,6017.456186,4.622787,POINT (1014594.110 215122.926),2551215,35
2551216,9380001047,1301.694419,6017.456186,4.622787,POINT (1014546.572 215118.217),2551216,34
2551217,9380001047,1301.694419,6017.456186,4.622787,POINT (1014499.033 215113.507),2551217,33
2551218,9380001047,1301.694419,6017.456186,4.622787,POINT (1014497.550 215124.242),2551218,33


In [57]:
sidewalk_nyc.merge(street_furniture_nyc, on='point_index', how='left', suffixes=('','_street_furniture'))
print(sidewalk_nyc.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2551220 entries, 0 to 2551219
Data columns (total 7 columns):
 #   Column        Dtype   
---  ------        -----   
 0   SOURCE_ID     int64   
 1   SHAPE_Leng    float64 
 2   SHAPE_Area    float64 
 3   SHAPE_Width   float64 
 4   geometry      geometry
 5   point_index   int64   
 6   ft_above_sea  uint16  
dtypes: float64(3), geometry(1), int64(2), uint16(1)
memory usage: 121.7 MB
None


### NYC Zoning (ZOLA)

In [58]:
zoning_nyc = gpd.read_file("data/nyc_zoning/nyzd.shp")

logger.success("ZOLA data loaded.")

2024-08-21 22:06:14 - robotability-score - SUCCESS - ZOLA data loaded.


In [59]:
# get the zoning of each point in sidewalk_nyc
sidewalk_nyc = gpd.sjoin(sidewalk_nyc, zoning_nyc, how='left', predicate='intersects').drop(columns=['index_right'])
print(sidewalk_nyc.info())


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2551220 entries, 0 to 2551219
Data columns (total 10 columns):
 #   Column        Dtype   
---  ------        -----   
 0   SOURCE_ID     int64   
 1   SHAPE_Leng    float64 
 2   SHAPE_Area    float64 
 3   SHAPE_Width   float64 
 4   geometry      geometry
 5   point_index   int64   
 6   ft_above_sea  uint16  
 7   ZONEDIST      object  
 8   Shape_Leng    float64 
 9   Shape_Area    float64 
dtypes: float64(5), geometry(1), int64(2), object(1), uint16(1)
memory usage: 199.5+ MB
None


### CitiBike Stations 
We simulate charging stations in the environment with existing CitiBike charging stations. It is also possible to simulate this via random sampling of the NYC road network, but we think citibike stations might be a more accurate distribution to pull from , as they are influenced by population density and zoning patterns. 

In [60]:
citibike_stations_nyc = pd.read_json("data/citibike/station_information.json")
citibike_stations_nyc = pd.json_normalize(citibike_stations_nyc['data']).T 
citibike_stations_nyc = pd.json_normalize(citibike_stations_nyc[0])
citibike_stations_nyc = gpd.GeoDataFrame(citibike_stations_nyc, geometry=gpd.points_from_xy(citibike_stations_nyc['lon'], citibike_stations_nyc['lat']), crs=WGS).to_crs(PROJ)

# get nearest station to each point in sidewalk nyc 
sidewalk_nyc = gpd.sjoin_nearest(sidewalk_nyc, citibike_stations_nyc, how='left').drop(columns=['index_right'])
print(sidewalk_nyc.info())

logger.success("Citibike stations data loaded.")

2024-08-21 22:06:52 - robotability-score - SUCCESS - Citibike stations data loaded.


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2551220 entries, 0 to 2551219
Data columns (total 19 columns):
 #   Column               Dtype   
---  ------               -----   
 0   SOURCE_ID            int64   
 1   SHAPE_Leng           float64 
 2   SHAPE_Area           float64 
 3   SHAPE_Width          float64 
 4   geometry             geometry
 5   point_index          int64   
 6   ft_above_sea         uint16  
 7   ZONEDIST             object  
 8   Shape_Leng           float64 
 9   Shape_Area           float64 
 10  lat                  float64 
 11  short_name           object  
 12  station_id           object  
 13  name                 object  
 14  region_id            object  
 15  capacity             int64   
 16  lon                  float64 
 17  rental_uris.android  object  
 18  rental_uris.ios      object  
dtypes: float64(7), geometry(1), int64(3), object(7), uint16(1)
memory usage: 374.7+ MB
None


### Street Lighting 

### Curb Ramps 

In [61]:
curb_ramps_nyc = pd.read_csv("data/pedestrian_curb_ramp_nyc.csv")
# prepend 'CURBRAMP' to each column name
curb_ramps_nyc.columns = [f'CURBRAMP_{col}' for col in curb_ramps_nyc.columns]
curb_ramps_nyc = gpd.GeoDataFrame(curb_ramps_nyc, geometry=wkt.loads(curb_ramps_nyc['CURBRAMP_the_geom']), crs=WGS).to_crs(PROJ)

In [72]:
# buffer the curb ramps and merge with sidewalk_nyc 
curb_ramps_nyc['geometry'] = curb_ramps_nyc['geometry'].buffer(50)
merged = gpd.sjoin(sidewalk_nyc, curb_ramps_nyc, how='left', predicate='intersects').drop(columns=['index_right'])
merged = merged.groupby('point_index')['CURBRAMP_RampID'].agg('count').to_frame().rename(columns={'CURBRAMP_RampID':'CURBRAMP_count'})

# groupby CURBRAMP_RampID and count the number of curb ramps in each group
sidewalk_nyc = sidewalk_nyc.merge(merged, left_on='point_index', right_index=True, how='left')
print(sidewalk_nyc.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2551220 entries, 0 to 2551219
Data columns (total 20 columns):
 #   Column               Dtype   
---  ------               -----   
 0   SOURCE_ID            int64   
 1   SHAPE_Leng           float64 
 2   SHAPE_Area           float64 
 3   SHAPE_Width          float64 
 4   geometry             geometry
 5   point_index          int64   
 6   ft_above_sea         uint16  
 7   ZONEDIST             object  
 8   Shape_Leng           float64 
 9   Shape_Area           float64 
 10  lat                  float64 
 11  short_name           object  
 12  station_id           object  
 13  name                 object  
 14  region_id            object  
 15  capacity             int64   
 16  lon                  float64 
 17  rental_uris.android  object  
 18  rental_uris.ios      object  
 19  CURBRAMP_count       int64   
dtypes: float64(7), geometry(1), int64(4), object(7), uint16(1)
memory usage: 394.2+ MB
None


### Raised Crosswalks 

In [25]:
raised_crosswalks_nyc = pd.read_csv("data/raised_crosswalks_nyc.csv")
raised_crosswalks_nyc.columns = [f'RAISEDCROSSWALK_{col}' for col in raised_crosswalks_nyc.columns]

In [73]:
raised_crosswalks_nyc = gpd.GeoDataFrame(raised_crosswalks_nyc, geometry=wkt.loads(raised_crosswalks_nyc['RAISEDCROSSWALK_WKT Geometry']), crs=PROJ)
raised_crosswalks_nyc['RAISEDCROSSWALK_index'] = raised_crosswalks_nyc.index
# prepend 'RAISEDCROSSWALK' to each column name
# buffer the raised crosswalks and merge with sidewalk_nyc
raised_crosswalks_nyc['geometry'] = raised_crosswalks_nyc['geometry'].buffer(50)


merged = gpd.sjoin(sidewalk_nyc, raised_crosswalks_nyc, how='left', predicate='intersects').drop(columns=['index_right'])
merged = merged.groupby('point_index')['RAISEDCROSSWALK_index'].agg('count').to_frame().rename(columns={'RAISEDCROSSWALK_index':'RAISEDCROSSWALK_count'})

# groupby RAISEDCROSSWALK_index and count the number of raised crosswalks in each group
sidewalk_nyc = sidewalk_nyc.merge(merged, left_on='point_index', right_index=True, how='left')

print(sidewalk_nyc.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2551220 entries, 0 to 2551219
Data columns (total 21 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   SOURCE_ID              int64   
 1   SHAPE_Leng             float64 
 2   SHAPE_Area             float64 
 3   SHAPE_Width            float64 
 4   geometry               geometry
 5   point_index            int64   
 6   ft_above_sea           uint16  
 7   ZONEDIST               object  
 8   Shape_Leng             float64 
 9   Shape_Area             float64 
 10  lat                    float64 
 11  short_name             object  
 12  station_id             object  
 13  name                   object  
 14  region_id              object  
 15  capacity               int64   
 16  lon                    float64 
 17  rental_uris.android    object  
 18  rental_uris.ios        object  
 19  CURBRAMP_count         int64   
 20  RAISEDCROSSWALK_count  int64   
dtypes: float64(7), geometry(1), 

### Sidewalk Scorecard 


In [74]:
sidewalk_scorecard_nyc = pd.read_csv("data/Scorecard_Ratings_20240814.csv") 
sidewalk_scorecard_nyc
sidewalk_scorecard_nyc.columns = [f'SCORECARD_{col}' for col in sidewalk_scorecard_nyc.columns]
# only keep 2023/09 
sidewalk_scorecard_nyc = sidewalk_scorecard_nyc[sidewalk_scorecard_nyc['SCORECARD_Month'] == '2023 / 09']

In [75]:
sidewalk_scorecard_nyc['cd_code'] = sidewalk_scorecard_nyc['SCORECARD_Community Board']
boro_mapping = { 
    'Manhattan': 1,
    'Bronx': 2,
    'Brooklyn': 3,
    'Queens': 4,
    'Staten Island': 5
}

sidewalk_scorecard_nyc['boro_code'] = sidewalk_scorecard_nyc['SCORECARD_Borough'].map(boro_mapping)

sidewalk_scorecard_nyc['cd_code'] = sidewalk_scorecard_nyc['cd_code'].astype(str)
sidewalk_scorecard_nyc['boro_code'] = sidewalk_scorecard_nyc['boro_code'].astype(str)

# prepend boro_code to cd_code, and padd cd_code so its always 2 digits 
sidewalk_scorecard_nyc['cd_code'] = sidewalk_scorecard_nyc['boro_code'] + sidewalk_scorecard_nyc['cd_code'].str.zfill(2)
sidewalk_scorecard_nyc.drop(columns=['SCORECARD_Month', 'SCORECARD_Borough', 'SCORECARD_Community Board', 'SCORECARD_District', 'SCORECARD_Cleaning Section', 'boro_code'], inplace=True)
# cast cd_code to int 
sidewalk_scorecard_nyc['cd_code'] = sidewalk_scorecard_nyc['cd_code'].astype(int)

# aggregate together rows with the same cd_code and mean 
sidewalk_scorecard_nyc = sidewalk_scorecard_nyc.groupby('cd_code').mean().reset_index()



cds_nyc = cds_nyc.merge(sidewalk_scorecard_nyc, left_on='boro_cd', right_on='cd_code', how='left')

In [76]:
# now, spatial join of cds_nyc with sidewalk_nyc
sidewalk_nyc = gpd.sjoin(sidewalk_nyc, cds_nyc, how='left', predicate='intersects').drop(columns=['index_right'])
print(sidewalk_nyc.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2551220 entries, 0 to 2551219
Data columns (total 33 columns):
 #   Column                                                      Dtype   
---  ------                                                      -----   
 0   SOURCE_ID                                                   int64   
 1   SHAPE_Leng                                                  float64 
 2   SHAPE_Area                                                  float64 
 3   SHAPE_Width                                                 float64 
 4   geometry                                                    geometry
 5   point_index                                                 int64   
 6   ft_above_sea                                                uint16  
 7   ZONEDIST                                                    object  
 8   Shape_Leng                                                  float64 
 9   Shape_Area                                                  float

### Points of Interest 

In [77]:
pois_nyc = pd.read_csv("data/pois_nyc.csv") 
# prepend 'POI' to each column name
pois_nyc.columns = [f'POI_{col}' for col in pois_nyc.columns]
pois_nyc = gpd.GeoDataFrame(pois_nyc, geometry=wkt.loads(pois_nyc['POI_the_geom']), crs=WGS).to_crs(PROJ)

In [78]:
# buffer the pois and merge with sidewalk_nyc
pois_nyc['geometry'] = pois_nyc['geometry'].buffer(50)
sidewalk_nyc = gpd.sjoin(sidewalk_nyc, pois_nyc, how='left', predicate='intersects').drop(columns=['index_right'])

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2553614 entries, 0 to 2551219
Data columns (total 49 columns):
 #   Column                                                      Dtype   
---  ------                                                      -----   
 0   SOURCE_ID                                                   int64   
 1   SHAPE_Leng                                                  float64 
 2   SHAPE_Area                                                  float64 
 3   SHAPE_Width                                                 float64 
 4   geometry                                                    geometry
 5   point_index                                                 int64   
 6   ft_above_sea                                                uint16  
 7   ZONEDIST                                                    object  
 8   Shape_Leng                                                  float64 
 9   Shape_Area                                                  float

### Traffic Density 

In [79]:
traffic_nyc = pd.read_csv("data/avg_traffic_by_sidewalk_august.csv", engine='pyarrow')

In [80]:
traffic_nyc = traffic_nyc[['point_index', '0','1','2']]
# columns = 'pedestrian', 'bike', 'car', 
traffic_nyc.columns = ['point_index', 'TRAFFIC_Pedestrian', 'TRAFFIC_Bike', 'TRAFFIC_Car']

In [81]:
# merge 
sidewalk_nyc = sidewalk_nyc.merge(traffic_nyc, on='point_index', how='left')

In [82]:
sidewalk_nyc

,SOURCE_ID,SHAPE_Leng,SHAPE_Area,SHAPE_Width,geometry,point_index,ft_above_sea,ZONEDIST,Shape_Leng,Shape_Area,...,POI_CREATED,POI_MODIFIED,POI_FACILITY_T,POI_SOURCE,POI_B7SC,POI_PRI_ADD,POI_NAME,TRAFFIC_Pedestrian,TRAFFIC_Bike,TRAFFIC_Car
0,2380000336,3504.153579,19782.066985,5.645320,POINT (1000940.908 165223.488),0,19,R4,16004.889786,6.129270e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.074074,0.000000,9.259259
1,2380000336,3504.153579,19782.066985,5.645320,POINT (1000948.337 165175.884),1,20,R4,16004.889786,6.129270e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040000,0.000000,8.920000
2,2380000336,3504.153579,19782.066985,5.645320,POINT (1000955.765 165128.280),2,20,R4,16004.889786,6.129270e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,8.812500
3,2380000336,3504.153579,19782.066985,5.645320,POINT (1000963.194 165080.677),3,19,R4,16004.889786,6.129270e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,9.000000
4,2380000336,3504.153579,19782.066985,5.645320,POINT (1000970.623 165033.073),4,20,R4,16004.889786,6.129270e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125000,0.000000,8.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553609,9380001047,1301.694419,6017.456186,4.622787,POINT (1014594.110 215122.926),2551215,35,R4,54234.666975,3.029753e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.567568,0.270270,5.783784
2553610,9380001047,1301.694419,6017.456186,4.622787,POINT (1014546.572 215118.217),2551216,34,R4,54234.666975,3.029753e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272727,0.227273,6.318182
2553611,9380001047,1301.694419,6017.456186,4.622787,POINT (1014499.033 215113.507),2551217,33,R4,54234.666975,3.029753e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,9.631579
2553612,9380001047,1301.694419,6017.456186,4.622787,POINT (1014497.550 215124.242),2551218,33,R4,54234.666975,3.029753e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,10.545455


### Shade 

In [37]:
# load treefolio data 
treefolio_nyc = gpd.read_file("data/treefolio84.geojson")

## Features that will not be modeled with empirical data 

### Weather Conditions 
We will assign weights to different types of weather. However, weather conditions may not be considered in use-cases where only the static built-environment is considered. 

### Existence of Detailed Digital Maps 

In the case of New York City, the entire city is covered by a DCM. As such, there is no need to bring this indicator in empirically, as the variance across the city is 0. 

## Features that cannot be modeled with empirical data 

### Sidewalk material 
There is no known dataset of sidewalk materials at the per-sidewalk level in NYC. Further, we deem that there is low variance; ie. the majority of sidewalks are all concrete. 

### Street lighting 
There is no known dataset of street lighting / lamp-posts in NYC. While a subset can be derived via the locations of relevant 311 complaints, we are unaware if this subset is representative of the overall distribution. 

## Features that are inferred via proxy datasets / other empirical data 


### Pedestrian flow 

### Sidewalk surface roughness 
We proxy this indicator via the sidewalk scorecard ratings dataset, which reports results at a granularity of the community district level.

### Shade 
We infer shade levels via the TreeFolio NYC dataset, which to-date only considers shade cast by street trees. There are other sources of shade in the city, namely tall buildings. 

## In-house indicators 

### Pedestrian density 
We compute the typical pedestrian densities at the per-sidewawlk level via YOLO (pretrained on COCO) inference on a large dataset of crowdsourced dashcam images. There are limitations. Namely, not every street segment in the city is covered: about 1/3 of our densely-sampled points along the sidewalk network lack data. Further, it is nontrivial & noisy to isolate which side of the street a pedestrian is walking on, and so there is noise in the map matching process. 

### Bike density
We compute bicycle densities using the same process as pedestrian density; and so, the same limitations carry over.  

### Vehicle density 
We compute vehicle densities using the same process as pedestrian density; and so, the same limitations carry over. 

In [38]:
sidewalk_nyc.to_csv("data/processed/score_dataset.csv")